In [15]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.ssa.*
import jline.solvers.mva.*
import jline.solvers.nc.*

GlobalConstants.setVerbose(VerboseLevel.STD)

In [16]:
val model = Network("model")

val n = 5  // number of items
val m = 2  // cache capacity

val source = Source(model, "Source")
val cacheNode = Cache(model, "Cache", n, m, ReplacementStrategy.RR)
val sink = Sink(model, "Sink")

val jobClass = OpenClass(model, "InitClass", 0)
val hitClass = OpenClass(model, "HitClass", 0)
val missClass = OpenClass(model, "MissClass", 0)

source.setArrival(jobClass, Exp(2.0))

// Zipf-like item references
val pAccess = Zipf(1.4, n)
cacheNode.setRead(jobClass, pAccess)

cacheNode.setHitClass(jobClass, hitClass)
cacheNode.setMissClass(jobClass, missClass)

val P = model.initRoutingMatrix()
P.set(jobClass, jobClass, source, cacheNode, 1.0)
P.set(hitClass, hitClass, cacheNode, sink, 1.0)
P.set(missClass, missClass, cacheNode, sink, 1.0)

model.link(P)

In [17]:
import jline.solvers.AvgTable

val solver = arrayOfNulls<Any>(4)
val avgTable = arrayOfNulls<AvgTable>(4)

solver[0] = CTMC(model, "keep", false, "cutoff", 1, "seed", 1)
avgTable[0] = (solver[0] as CTMC).avgNodeTable
avgTable[0]!!.print()

CTMC analysis [method: default, lang: java, env: 17.0.15] completed in 0.063732s.
Station  JobClass   QLen        Util        RespT       ResidT      ArvR        Tput        
--------------------------------------------------------------------------------------------
Source   InitClass  0.00000     0.00000     0.00000     0.00000     0.00000     2.00000     
Cache    InitClass  0.00000     0.00000     0.00000     0.00000     2.00000     0.00000     
Cache    HitClass   0.00000     0.00000     0.00000     0.00000     0.00000     2.00000     
Sink     HitClass   0.00000     0.00000     0.00000     0.00000     2.00000     0.00000     
--------------------------------------------------------------------------------------------


In [18]:
model.reset()
solver[1] = SSA(model, "samples", 10000, "verbose", true, "method", "serial", "seed", 1)
avgTable[1] = (solver[1] as SSA).avgNodeTable
avgTable[1]!!.print()

SSA samples:   10000
SSA analysis [method: serial, lang: java, env: 17.0.15] completed in 1.113519s. Iterations: 10001.
Station  JobClass   QLen        Util        RespT       ResidT      ArvR        Tput        
--------------------------------------------------------------------------------------------
Source   InitClass  0.00000     0.00000     0.00000     0.00000     0.00000     2.00000     
Cache    InitClass  0.00000     0.00000     0.00000     0.00000     2.00000     0.00000     
Cache    HitClass   0.00000     0.00000     0.00000     0.00000     0.00000     1.16331     
Cache    MissClass  0.00000     0.00000     0.00000     0.00000     0.00000     0.83669     
Sink     HitClass   0.00000     0.00000     0.00000     0.00000     1.16331     0.00000     
Sink     MissClass  0.00000     0.00000     0.00000     0.00000     0.83669     0.00000     
--------------------------------------------------------------------------------------------


In [19]:
model.reset()
solver[2] = MVA(model, "seed", 1)
avgTable[2] = (solver[2] as MVA).avgNodeTable
avgTable[2]!!.print()

MVA analysis [method: default(fpi), lang: java, env: 17.0.15] completed in 0.001260s.
Station  JobClass   QLen        Util        RespT       ResidT      ArvR        Tput        
--------------------------------------------------------------------------------------------
Source   InitClass  0.00000     0.00000     0.00000     0.00000     0.00000     2.00000     
Cache    InitClass  0.00000     0.00000     0.00000     0.00000     2.00000     0.00000     
Cache    HitClass   0.00000     0.00000     0.00000     0.00000     0.00000     1.11270     
Cache    MissClass  0.00000     0.00000     0.00000     0.00000     0.00000     0.88730     
Sink     HitClass   0.00000     0.00000     0.00000     0.00000     1.11270     0.00000     
Sink     MissClass  0.00000     0.00000     0.00000     0.00000     0.88730     0.00000     
--------------------------------------------------------------------------------------------


In [20]:
model.reset()
solver[3] = NC(model, "seed", 1)
avgTable[3] = (solver[3] as NC).avgNodeTable
avgTable[3]!!.print()

val hitRatio = cacheNode.hitRatio
val missRatio = cacheNode.missRatio
println("hitRatio = $hitRatio")
println("missRatio = $missRatio")

NC analysis [method: default(rayint), lang: java, env: 17.0.15] completed in 0.001587s. Iterations: 100000.
Station  JobClass   QLen        Util        RespT       ResidT      ArvR        Tput        
--------------------------------------------------------------------------------------------
Source   InitClass  0.00000     0.00000     0.00000     0.00000     0.00000     2.00000     
Cache    InitClass  0.00000     0.00000     0.00000     0.00000     2.00000     0.00000     
Cache    HitClass   0.00000     0.00000     0.00000     0.00000     0.00000     1.17218     
Cache    MissClass  0.00000     0.00000     0.00000     0.00000     0.00000     0.82782     
Sink     HitClass   0.00000     0.00000     0.00000     0.00000     1.17218     0.00000     
Sink     MissClass  0.00000     0.00000     0.00000     0.00000     0.82782     0.00000     
--------------------------------------------------------------------------------------------
hitRatio = [ 0.5860916349276919   0.0   0.0 ]

missRati